## Analiza danych

#### Przegląd atrybutów z *listings*

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

sys.path.append(str(Path("..").resolve()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

listings = pd.read_csv("../data/raw/listings.csv")
listings.info()

ModuleNotFoundError: No module named 'seaborn'

#### Wybór atrybutów 
Do obliczenia przewidywanej ceny wymagane są przede wszystkim kolumny definiujące standard kwatery. 
Atrybuty związane z dostępnością, cechami gospodarza lub opiniami użytkowników mogą także mieć wpływ na zmienną objaśnianą, dlatego póki co nie zostają usunięte (jeśli późniejsza analiza wskaże, że są wtórne zostaną wykluczone). Dodatkowo można pozbyć się kolumn zawierających metadane, ponieważ są one całkowicie nieinformatywne.

Lista wybranych początkowo atrybutów znajduje się w pliku *features.py*, a zastosowanych transformacji w *transformations/listings.py*.

In [ ]:
from src.features import INITIAL_FEATURES,TARGET
from src.transformations.listings import *

target = listings[TARGET]
listings = listings[INITIAL_FEATURES] 
listings.info()

#### Atrybuty nienumeryczne

In [ ]:
object_columns = listings.select_dtypes(include=["object"]).columns
for col in object_columns:
    print(f"{col}: {listings[col][0]}")

#### *property_type* i *room_type*

In [ ]:
listings["property_type"].isna().sum()

In [ ]:
listings["property_type"].value_counts()

W celu ograniczenia szumu wynikającego z wielu wartości w kolumnie property type, wartości zostały zgeneralizowane do kilku podstawowych kategorii. Pondato został dodany nowy atrybut (*is_luxury*), który wskazuje czy dane lokum jest luksusowe. Taki atrybut może być silnie skorelowany z ceną.

In [ ]:
listings = add_is_luxury_attribute(listings)
listings["is_luxury"].value_counts()

In [ ]:
listings = aggregate_property_type(listings)
listings["property_type"].value_counts().plot(kind="pie", autopct="%1.0f%%")

Atrybut *room_type* pozostaje bez zmian, jest już wystarczająco dobrze podzielony na podgrupy. 

In [ ]:
listings["room_type"].isna().sum()

In [ ]:
listings["room_type"].value_counts()

#### *bathrooms_text* i *bathrooms*

Puste wartości w *bathrooms* zostały uzupełnione biorąc dane z *bathrooms_text* - tam braków jest zdecydowanie mniej. Ponadto dodano nowy atrybut *is_bathroom_shared*, który został wyłuskany z tekstu. Po przetworzeniu danych w *bathroom_text* ten atrybut może zostać całkowicie usunięty, nie niesie żadnej nowej informacji.

In [ ]:
print(listings["bathrooms_text"].isna().sum())
print(listings["bathrooms"].isna().sum())

In [ ]:
listings = fill_bathrooms_values_from_text(listings)
print(listings["bathrooms_text"].isna().sum())
print(listings["bathrooms"].isna().sum())

In [ ]:
listings = add_is_bathroom_shared_attribute(listings)
listings["is_bathroom_shared"].value_counts()

#### *amenities*

In [ ]:
listings["amenities"].isna().sum()

In [ ]:
listings["amenities"].head(10)

In [ ]:
from src.amenities_correlation import get_amenities_counter

counter = get_amenities_counter(listings)
unique = [a for a, _ in counter.items()]
print(f'Unique amenities: {len(unique)}')

In [ ]:
top_20 = counter.most_common(20)
amenities, counts = zip(*top_20)

plt.figure(figsize=(10,8))
bars = plt.barh(amenities[::-1], counts[::-1], color="#86bf91")
for bar in bars:
    width = bar.get_width()
    y = bar.get_y() + bar.get_height() / 2
    plt.text(width, y, str(width),  va="center", ha="right")
plt.grid(axis="x", linestyle="--", alpha=0.5)
plt.xlabel("Number of Listings")
plt.title("Top 20 Most Common Amenities")
plt.show()

Z uwagi na bardzo dużą liczbę unikalnych udogodnień zostanie wybranych 10, które:
- wyglądają wymiernie (ocena subiektywna);
- mają dużą korelację Spearmana z kolumną price;
- występują w minimum 50 obiektach.

In [ ]:
from src.amenities_correlation import * 
from src.transformations.target import * 

converted_target = convert_price_to_number(target)

amenities_correlation_df = calc_amenities_correlation(listings.copy(), converted_target, min_freq=50)
amenities_mutual_info_df = calc_amenities_mutual_info(listings.copy(), converted_target, min_freq=50)["mutual_info"]

merged = (
    amenities_correlation_df.rename("pearson")
    .to_frame()
    .join(amenities_mutual_info_df.rename("mutual_info"), how="inner")
    .sort_values("pearson", ascending=False)
)

merged.head(n=15)

In [ ]:
amenities_correlation_df = amenities_correlation_df.abs()

top_15 = amenities_correlation_df.sort_values(ascending=False).head(15)
ax = top_15.sort_values(ascending=True).plot(
    kind='barh',
    figsize=(10, 8),
    color='#FF5A5F',
    title='Top 15 amenities correlating with price (absolute values)'
)
for bar in ax.patches:
    width = bar.get_width()
    y = bar.get_y() + bar.get_height() / 2
    plt.text(width, y, f"{width:.5f}",  va="center", ha="right")
plt.xlabel('Correlation with price')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

Finalnie wybrane udogodnienia to:

In [ ]:
from src.features import AMENITIES
AMENITIES

Zostaną one zakodowane binarnie

In [ ]:
listings = encode_amenities_binary(listings, AMENITIES)
filtered = listings.filter(like="amenity")
filtered.info()

#### *description* i *neighborhood_overview*
W przypadku kolumn zawierających opisy tekstowe zostanie wyznaczonych ich sentyment. Często opisy zawierają kluczowe informacje dotyczące standardu mieszkania czy też atrakcyjności okolicy, których nie da się uwzględnić w innych atrybutach. Przed wyliczaniem atrybutu dane tekstowe są czyszczone z tagów html. Wartości puste zostaną później uzupełnione w pipelinie ml. 

In [ ]:
listings["description"].isna().sum()

In [ ]:
listings["description"].head()

In [ ]:
listings = convert_description_to_sentiment(listings)
listings["description_sentiment"].describe()

In [ ]:
hist = listings.hist(bins=25, column="description_sentiment", color="#86bf91", zorder=2, rwidth=0.9)

In [ ]:
listings["neighborhood_overview"].isna().sum()

In [ ]:
listings = convert_neighborhood_overview_to_sentiment(listings)
listings["neighborhood_overview_sentiment"].describe()

In [ ]:
hist = listings.hist(bins=25, column="neighborhood_overview_sentiment", color="#86bf91", zorder=2, rwidth=0.9)

Rozkłady sentymentów okazały się zaskakujące. Wbrew wstępnym założeniom, zgodnie z którymi wystawiający oferty powinni wykazywać tendencję do nadmiernie pozytywnego opisywania swoich lokali, analizowany atrybut charakteryzuje się rozkładem zbliżonym do neutralnego.

#### *host_response_time*
W przypadku tego atrybutu można dostrzec, że ma on natrualny porządek tzn. *within an hour* jest większe/lepsze niż *within a few hours* - na etapie pipelinu ml można stosować kodowanie polegające na przypisaniu kolejnych liczb całkowitych.

In [ ]:
listings["host_response_time"].isna().sum()

In [ ]:
listings["host_response_time"].value_counts().plot(kind="pie")

#### *host_response_rate* i *host_acceptance_rate*
Te atrybuty wymagają zamienienia łańcuchów znaków reperezentujących procenty na liczby zmiennoprzecinkowe.

In [ ]:
listings["host_response_rate"].head()

In [ ]:
columns_to_convert = ["host_response_rate", "host_acceptance_rate"]
listings = convert_percentage_columns(listings, columns_to_convert)

for col in columns_to_convert:
    print(f"{col} example value: {listings[col][0]}")


#### *host_is_superhost*, *host_identity_verifed* i *instant_bookable*
Powyższe atrybuty zamiast wartości binarnych są reprezentowane przez *"t" i "f"* - muszą zostać zamienione na wartości liczbowe, żeby model był w stanie je obsłużyć.

In [ ]:
listings["host_is_superhost"].head()

In [ ]:
columns_to_convert = ["host_is_superhost", "host_identity_verified", "instant_bookable"]
listings = convert_tf_columns(listings, columns_to_convert)

for col in columns_to_convert:
    print(f"{col} example value: {listings[col][0]}")

#### Przegląd atrybutów z *sessions*

In [ ]:
from src.transformations.sessions import *

sessions = pd.read_csv("../data/raw/sessions.csv")
sessions.info()

#### Wybór atrybutów 
Do utworzenia nowych, zagregowanych kolumną posłużą wszystkie atrybuty. Nowe kolumnę będą reprezentowały zachowanie użytkowników mogące wskazywać na realny popyt, który powinien mieć wpływ na cenę. (później zostanie przeprowadzona analiza wtórna, aby ewentualnie wykluczyć zbędnie utworzone kolumny)

Lista zastosowanych transformacji znajduje się w *transformations/sessions.py*.

#### Czyszczenie wstępne
- *action* jako kolumna decyzyjna

In [ ]:
sessions["action"].value_counts()

Rekordów, dla których akcja to *browse_listings*, nie da się połączyć z konkretnymi *listing_id*, dlatego zostaną one odrzucone. 

In [ ]:
sessions = drop_browse_listings(sessions)
sessions["action"].value_counts()

- rekordy starsze niż 1 rok

Sytuacja na rynku wynajmu krótkoterminowego jest bardzo dynamiczna, dlatego aby ograniczyć dryf danych oraz ich zaszumienie, na wstępnie odrzucone zostaną rekordy starsze niż **1 rok**. 

In [ ]:
sessions = convert_timestamps_to_dates(sessions)
sessions["timestamp"].max()

In [ ]:
sessions = drop_records_older_than_one_year(sessions)
sessions["action"].value_counts()

- rozkład danych w ostatnim roku

In [ ]:
sessions["timestamp"] = pd.to_datetime(sessions["timestamp"], errors="coerce")
valid_dates = sessions["timestamp"].dropna()

plt.figure(figsize=(15, 6))

plt.hist(valid_dates, bins=100, color='skyblue', edgecolor='black')

plt.title("Rozkład Timestampów w Danych")
plt.xlabel("Data")
plt.ylabel("Liczba zdarzeń")
plt.grid(axis='y', alpha=0.5)

plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

Ze względu na znaczną dysproporcję w ilości danych między poszczególnymi miesiącami, wszystkie cechy zostaną zagregowane w ujęciu **rocznym**.

#### *views_ltm*

Liczba wyświetleń oferty w ostatnim roku. Kolumna świadcząca o popycie (tylko rekordy, dla których *action* == *view_listing*)

In [ ]:
listings_stats = get_views_last(sessions)
listings_stats.head()

In [ ]:
print(f'Średnia liczba odsłon oferty w ciągu ostatniego roku: {listings_stats["listing_views_ltm"].mean():.2f}')

#### *unique_viewers*

Liczba unikalnych oglądających danej oferty. Służy wykryciu sytuacji, gdy ograniczona ilość użytkowników produkuje wiele "pustych" wyświetleń.

In [ ]:
listings_stats = get_unique_viewers_last(sessions, listings_stats)
listings_stats.head()

In [ ]:
print(f'Średnia liczba unikalnych widzów oferty w ciągu ostatniego roku: {listings_stats["unique_viewers_ltm"].mean():.2f}')

#### *conversion_rate* (*bookings* / *views*)

Jaka część wyświetleń przełożyła się na faktyczną rezerwację oferty. Atrybut w zakresie <0, 1>.

In [ ]:
listings_stats = get_conversion_rate(sessions, listings_stats)
listings_stats.head()

In [ ]:
print(f'Średnia konwersja oferty: {listings_stats["conversion_rate_ltm"].mean():.2f}')

#### *average_lead_time*

Średnie wyprzedzenie, z jakim dokonywana była rezerwacja. Wskazuje, czy pobyt był planowany z wyprzedzeniem, czy była to rezerwacja "last minute". W przypadku gdy lokal nie miał żadnej rezerawcji (*conversion_rate* = 0) *avg_lead_time* jest nieokreślone. Dopiero na poziomie pipeline'u ml zostanie imputowane.

In [ ]:
listings_stats = get_average_lead_time(sessions, listings_stats)
listings_stats.head()

In [ ]:

print(f'Średni średni czas wyprzedzenia (lead time) oferty: {listings_stats["average_lead_time"].mean():.2f} dni')

#### *average_booking_duration*

Średnia liczba dni pojedynczej rezerwacji. Rozróżnia pobyty długoterminowe, od krótkoterminowych. Wartości nieokreślone występują analogicznie jak w przypadku atrybutu *average_lead_time*.

In [ ]:
listings_stats = get_average_booking_duration(sessions, listings_stats)
listings_stats.head()

In [ ]:
print(f'Średnia średnia długość pobytu (booking duration) oferty: {listings_stats["average_booking_duration"].mean():.2f} dni')

#### Połączenie wszystkich atrybutów w jeden zbiór

Najpierw odrzucimy kolumny, które zostały już przetworzone.

In [ ]:
drop = ["bathrooms_text", "description", "neighborhood_overview", "amenities"]
listings.drop(columns=drop, inplace=True)

In [ ]:
features = listings.merge(listings_stats, left_on="id", right_on="listing_id", how="left")
drop = ["listing_id", "id"]
features.drop(columns=drop, inplace=True)

In [ ]:
features.info()

In [ ]:
numeric_cols = features.select_dtypes(include=["float64", "int64", "Int64"]).columns
num_df = features[numeric_cols].astype(float)
corr_matrix = num_df.corr()
plt.figure(figsize=(14,12))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    cbar=True,
    square=True,
    linewidths=.5
)
plt.title("Macierz korelacji (Pearson) - cechy liczbowe i binarne", fontsize=16)
plt.show()

### Target - *price*

In [ ]:
target.isna().sum()

In [ ]:
target.head()

In [ ]:
target = convert_price_to_number(target)
target.head()

In [ ]:
hist = target.hist(bins=25, color="#86bf91", zorder=2, rwidth=0.9)

W celu minimalizacji znaczenia outlinerów (ceny mocno odbiegające od średniej) zastosowano skalowanie logarytmiczne, które ustabilizuje proces uczenia.

In [ ]:
target = logarithmize_price(target)
hist = target.hist(bins=25, color="#86bf91", zorder=2, rwidth=0.9)

In [ ]:
target.describe()